In [1]:
import pandas as pd
import numpy as np
import hashlib as hl
import matplotlib.pyplot as plt
%matplotlib inline

### Loading the Data File:

In [2]:
data = pd.read_csv('/Users/narayani/Desktop/IIITB/Sem6/ML/ML_Project/Housing_Linear Regression/data/housing.csv')
#data.head()

In [3]:
'''def splitTrainData(total_data, ratio):
    indices_Shuffled = np.random.permutation(len(total_data))
    test_data_size = int(len(total_data)* ratio)
    indices_Test = indices_Shuffled[:test_data_size]
    indices_Train = indices_Shuffled[test_data_size:]
    return total_data.iloc[indices_Train], total_data.iloc[indices_Test]
training_data, test_data = splitTrainData(data, 0.3)'''

from sklearn.model_selection import train_test_split
training_set, test_set = train_test_split(data,test_size=0.2,random_state = 42)
#print("Train: ",len(train_set), " +  Test: " , len(test_set))

In [4]:

# X_training = training_data.drop(['median_house_value', 'ocean_proximity'],axis =1)
# Y_labels = training_data["median_house_value"].copy()

# from sklearn.preprocessing import Imputer
# imputer = Imputer(strategy = "median")

# imputer.fit(X_training)
# #imputer.statistics_
# #X_training.median().values

# X_imp = imputer.transform(X_training)
# X_inp = pd.DataFrame(X_imp, columns = X_training.columns)

# X_data = X_inp.as_matrix()
# Y = Y_labels.as_matrix()

# X = np.c_[np.ones((len(X_inp),1)),X_data]
# #X= (X - X.mean())/(X+X.std())
# theta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(Y)

# theta

### Pre-Process the data:
                To remove "NAN-Values" wherever there in the data. And Since, column named "Ocean Proximity" is a text field, we have encode it, so that it can be used for ML-Algorithm.

In [5]:
def preprocessData ( train_set ):
#Removing the target values and copying them as labels/target value for feature values of each data-point.
#new data is the data without the target values
    new_data = train_set.drop("median_house_value", axis = 1)
    labels = train_set["median_house_value"].copy()
    
#Imputer is used to remove all the NAN values in the dataset, but after the removal of the text field i.e "ocean_proximity"
    from sklearn.preprocessing import Imputer
    imputer = Imputer(strategy = "median")  
    ntx_data = new_data.drop("ocean_proximity", axis =1)
    imputer.fit(ntx_data)
    ntx_data_imp = imputer.transform(ntx_data)
    
#Converting into Pandas-Dataframe. 
    ntx_data_inp = pd.DataFrame(ntx_data_imp, columns = ntx_data.columns,index = list(ntx_data.index.values))
    
#Encoding the text-field "ocean_proximity".
    housing_cat = new_data['ocean_proximity']
    from sklearn.preprocessing import OneHotEncoder
    housing_cat_encoded, housing_categories = housing_cat.factorize()
    encoder = OneHotEncoder()
#Reshaping housing_cat_encoder as it is 1-D array, but fit_transform expects a 2-D array.
    housing_cat_1hot =encoder.fit_transform(housing_cat_encoded.reshape(-1,1))
    (housing_cat_1hot.toarray())

    import category_encoders as c_enc
    encoder = c_enc.OneHotEncoder()
    housing_cat_reshaped = housing_cat.values.reshape(-1,1)
    encoder.fit(housing_cat_reshaped)
    
    X_cleaned = encoder.transform(housing_cat_reshaped)
#Converting Encoded values for "ocean_proximity" to matrix
    enc_data = X_cleaned.as_matrix()
    
#Feature Scaling for data without the encoded values for "ocean_proximity":
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(ntx_data_inp)
    data_without_enc = scaler.transform(ntx_data_inp)

#data without encoded values is being appended by the encoded values to get the complete data for computation.
    data_training = np.append(data_without_enc, enc_data, axis = 1)
    data_training = X = np.c_[np.ones((len(data_training),1)),data_training]
    
    
    return data_training, labels
    
    
    
    
    

### Training on the Training Set:

In [6]:
training_data , training_labels = preprocessData(training_set)
testing_data , testing_labels = preprocessData(test_set)

In [7]:
# from sklearn.linear_model import LinearRegression
# lin_reg = LinearRegression()
# lin_reg.fit(training_data, training_labels)
# lin_reg.predict(training_data)

### Parameter matrix from Normal Equation:

In [8]:
#temp = (training_data.T).dot(training_data)

theta = np.linalg.pinv(training_data.T.dot(training_data)).dot(training_data.T).dot(training_labels)
theta

array([199021.96620438, -53826.64801649, -54415.6961445 ,  13889.86618856,
       -13094.25116219,  43068.18184187, -43403.43242732,  18382.19632373,
        75167.77476625,  24308.95045207, -18908.84578241,  20877.81037892,
        15741.16816144, 157002.88299432,      0.        ])

### Predicted labels for training data:

In [9]:
train_predicted_Y = training_data.dot(theta.T)
train_predicted_Y

array([188628.07724361, 290379.8948687 , 250985.48476349, ...,
       194624.05524513, 281818.52422053, 271572.8418584 ])

In [10]:
# X_test = test_data.drop(['median_house_value','ocean_proximity'], axis =1)
# Y_actuals = test_data["median_house_value"].copy()

# imputer.fit(X_test)
# X_test_imp = imputer.transform(X_test)
# test_inp = pd.DataFrame(X_test_imp, columns = X_test.columns)

# data_test = test_inp.as_matrix()
# actual_Y = Y_actuals.as_matrix()

# #test_inp.info()
# actual_Y

#test = np.c_[np.ones((len(test_inp)),1),data_test]
#test = np.c_[np.ones((len(test_inp),1)),data_test]
#test



### Predicated labels for test data:

In [11]:
test_predicted_Y = testing_data.dot(theta.T)
test_predicted_Y


array([ 98165.82772806, 168705.1233016 , 223015.48407995, ...,
       444695.41566959, 166431.68590505, 185544.84958196])

In [12]:
# X_housing = data.drop(['median_house_value', 'ocean_proximity'],axis =1)
# Y_housing = data["median_house_value"].copy()

# imputer.fit(X_housing)

# X_housing_imp = imputer.transform(X_housing)
# X_housing_inp = pd.DataFrame(X_housing_imp, columns = X_housing.columns)

# X_housing_data = X_housing_inp.as_matrix()
# housing_Y = Y_housing.as_matrix()
# housing = np.c_[np.ones((len(X_housing_inp),1)),X_housing_data]



### Training on the Complete Data:

In [13]:
housing_data , housing_labels = preprocessData(data)

In [14]:
#Calulating new Parameter Matrix for the complete dataset
theta_new = np.linalg.pinv(housing_data.T.dot(housing_data)).dot(housing_data.T).dot(housing_labels)
theta_new

array([202257.61270761, -52952.95152846, -53767.62485624,  13312.88334575,
       -10320.06092603,  29920.76507621, -44490.47744263,  29746.22226671,
        73636.15586366,  13281.98627461,  16979.3879357 , -22787.0108087 ,
        21738.14154797, 173045.10775805,      0.        ])

### Predicated labels for whole housing dataset without splitting:

In [15]:
housing_predicted_Y = housing_data.dot(theta_new.T)

housing_predicted_Y

array([408492.35829822, 423996.66388553, 378466.63041436, ...,
        40399.753241  ,  50937.88670863,  67523.43037025])

### Root Mean Squared Error :

In [16]:
from sklearn.metrics import mean_squared_error

test_mse = mean_squared_error(testing_labels, test_predicted_Y)
train_mse = mean_squared_error(training_labels,train_predicted_Y)
housing_mse = mean_squared_error(housing_labels, housing_predicted_Y)


print "RMSE for Traning Data: "+ str(np.sqrt(train_mse))
print "RMSE for Test Data: " + str(np.sqrt(test_mse))
print "RMSE for Housing Data taken without splitting: "+ str(np.sqrt(housing_mse))

RMSE for Traning Data: 68433.93736666226
RMSE for Test Data: 75275.2554019648
RMSE for Housing Data taken without splitting: 68709.32557762168
